In [36]:
import pandas as pd
import os
from os.path import dirname
import SimpleITK as sitk
import numpy as np
import itk
import matplotlib.pyplot as plt
import re
modifyDir = "modified_data"
dense_compact_seg_file_name = "TRIO_TRIO_MR2_dense_seg.nii.gz"
newImageName = "TRIO_TRIO_MR2_dense_seg_LeftRightIndependent.nii.gz"
lookupTableFileName = "BAWHDAdultAtlas_FreeSurferConventionColorLUT_20160524.txt"
newlookupTableFileName = "BAWHDAdultAtlas_FreeSurferConventionColorLUT_LeftRightIndependent.txt"

In [40]:
# Get the path of the lookup table to be modified
lookupTablePath = os.path.join(dirname(os.getcwd()), modifyDir, lookupTableFileName)
print(lookupTablePath)
print(os.getcwd())
# Create a data frame from the file containing the lookup table
df = pd.read_table(lookupTablePath, error_bad_lines=False, comment="#")

/Shared/johnsonhj/2018Projects/20181112_RecodeLabelMaps/modified_data/BAWHDAdultAtlas_FreeSurferConventionColorLUT_20160524.txt
/Shared/johnsonhj/2018Projects/20181112_RecodeLabelMaps/src


Skipping line 1083: expected 6 fields, saw 7
Skipping line 1089: expected 6 fields, saw 7
Skipping line 1092: expected 6 fields, saw 7



In [32]:
origIndexByLabel = {}
# Create a dictionary associating a Label with a Number
origIndexByLabel = dict(zip(df['Label'], df['No.']))
oldLabelList = df.index.tolist()


newLabelList = []
indexByLabel = {}
newIndexByOldIndex = {}

# Remove all right or left indications in the dataframe
# for index, label in enumerate(df['Label']):
for index, row in df.iterrows():
    label = row['Label']
    label = re.sub(r'Left|Right|left|right|lh|rh|', '', label)
    label = re.sub(r'^L[A-Z].*|^R[A-Z].*', label[1:], label)
    label = re.sub(r'^-', '', label)
    
    if label not in indexByLabel:
        # If a right or left indicator was removed from a label, rewrite the label
        newIndexByOldIndex[row['No.']] = row['No.']
        indexByLabel[label] = row['No.']
    else:
        newIndexByOldIndex[row['No.']] = indexByLabel[label]
    newLabelList.append(label)
       
    
# Rewrite the label column with the right and left indicators removed
df['Label'] = newLabelList
# Rewrite the number for labels so that the right and left of a label are identified by the same number
df['No.'] = newIndexByOldIndex.values()

In [33]:
# Make a file for the new lookup table
newLookupTablePath = os.path.join(dirname(os.getcwd()), modifyDir, newlookupTableFileName)
# Create the dataframe for the new file
df.to_csv(newLookupTablePath, sep='\t', index=False)

In [34]:
# Get the image for which the labels will be rewritten
dense_compact_path = os.path.join(dirname(os.getcwd()), modifyDir, dense_compact_seg_file_name)
image = sitk.ReadImage(dense_compact_path)

In [35]:
# Alter the image so that its labels are referenced only by the new indices (get rid of half of the indices)
imageFilter = sitk.ChangeLabelImageFilter()
newImage = imageFilter.Execute(image, newIndexByOldIndex)
newImagePath = os.path.join(dirname(os.getcwd()), modifyDir, newImageName)
sitk.WriteImage(newImage, newImagePath)